In [1]:
from pyiron_workflow import Workflow
from pyironflow.pyironflow import PyironFlow
from typing import Optional

In [2]:
from ob.openbis import openbis_login

In [3]:
o = openbis_login("https://test3.datastore.bam.de/", "pkruziko")

Enter openBIS password:  ········


In [3]:
@Workflow.wrap.as_function_node
def CreateProject(pr_name:str):
    from pyiron_atomistics import Project
    pr = Project(pr_name)
    return pr

In [4]:
@Workflow.wrap.as_function_node
def CreateStructure(pr, structure_name_prefix: str, element:str, bravais_lattice: Optional[str], a: Optional[float|int], c: Optional[float|int] = None, cubic:bool = False):
    pr = pr
    if bravais_lattice != 'hcp' and c == None:
        structure = pr.create.structure.bulk(element, crystalstructure=bravais_lattice, cubic=cubic, a=a)
    elif bravais_lattice == 'hcp' and c == None:
        structure = pr.create.structure.bulk(element, crystalstructure=bravais_lattice, cubic=cubic, a=a)
    else:
        structure = pr.create.structure.bulk(element, crystalstructure=bravais_lattice, cubic=cubic, a=a, c=c)

    from pyiron_base.storage.hdfio import FileHDFio
    hdf = FileHDFio(pr.name + '/' + structure_name_prefix + '_input_structure.h5')
    structure.to_hdf(hdf)
    
    return structure

In [5]:
@Workflow.wrap.as_function_node("View")
def VisualizeStructure(structure):
    return structure.plot3d

In [6]:
@Workflow.wrap.as_function_node
def LammpsCalcMinimize(pr, structure, job_name:str, pot_list_index: int, f_tol: float, min_style: str, del_ex_job: bool = False, is_child_job: bool = False):
    job = pr.create.job.Lammps(job_name, delete_existing_job=del_ex_job)
    job.structure = structure.copy()
    job.potential = job.list_potentials()[pot_list_index]
    job.calc_minimize(f_tol=f_tol, style=min_style)
    job.run()

    if not is_child_job:
        import os
        import platform
        if "Windows" in platform.system():
            os.system('conda env export | findstr /V "^prefix: " > ' + job.path + '_environment.yml')
        else:
            os.system('conda env export | grep -v "^prefix: " > ' + job.path + '_environment.yml')
        
        from pyiron_base.storage.hdfio import FileHDFio
        hdf = FileHDFio(job.path + '_input_structure.h5')
        job.structure.to_hdf(hdf)
        
    return job

In [7]:
@Workflow.wrap.as_function_node("object")
def LammpsJobOpenBIS(o, space:str, project:str, collection:str, job, separate_input_sample: bool = True):
    from ob.concept_dict import process_lammps_job, process_structure_crystal
    from ob.openbis import GenericLammpsJobObject, GenericCrystalObject
    cdict = process_lammps_job(job)
    struct_cdict = process_structure_crystal(job)

    from IPython.display import display
    if separate_input_sample == True:
        s = GenericCrystalObject(o, space, project, collection, "input", struct_cdict)
        j = GenericLammpsJobObject(o, space, project, collection, cdict, struct_cdict)
        display(s)
        display(j)
    else:
        j = GenericLammpsJobObject(o, space, project, collection, cdict)
        display(j)

    # openbis logout where ??
            
    return "Done"

In [11]:
wf = Workflow("test_wf")

wf.Project = CreateProject("test_complete")
wf.Structure = CreateStructure(pr=wf.Project, structure_name_prefix="", element="Fe", bravais_lattice="bcc", a=2.845, cubic=True)
wf.VisualizeStructure = VisualizeStructure(wf.Structure)
#wf.SampleKG = SampleKG(wf.Project_KG_Element)
wf.LammpsCalcMinimize = LammpsCalcMinimize(wf.Project, wf.Structure, "test_lammps_gui", 68, 1e-8, "fire", True, True)
#wf.JobKG = JobKG(wf.Project_KG_Element, wf.LammpsCalcMinimize)
#wf.LammpsJobOpenBIS = LammpsJobOpenBIS(o, "PKRUZIKO", "TEST", "TEST_EXP_1", wf.LammpsCalcMinimize)

In [12]:
wf.LammpsCalcMinimize.pull()

/home/tlakshmi/anaconda3/envs/pyiron_unify_2/lib/python3.11/site-packages/pyiron_atomistics/lammps/interactive.py:302: UserWarning: f_tol is deprecated as of vers. 0.3.0. It is not guaranteed to be in service in vers. 0.4.0. Use ionic_force_tolerance instead.
  warnings.warn(


The job test_lammps_gui was saved and received the ID: 2422


{'groups': ['input', 'output'], 'nodes': ['DICT_VERSION', 'HDF_VERSION', 'NAME', 'OBJECT', 'TYPE', 'VERSION', 'executable', 'job_id', 'server', 'status']}

In [ ]:
pf = PyironFlow([wf])
pf.gui

In [11]:
o.logout()